# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f51adf575e0>
├── 'a' --> tensor([[ 2.0032,  0.9398,  0.1247],
│                   [-0.3039,  0.9653,  0.4137]])
└── 'x' --> <FastTreeValue 0x7f51adf57160>
    └── 'c' --> tensor([[-0.4802, -0.4363,  1.1402, -0.0180],
                        [-1.7043, -1.2722, -1.3475, -1.8720],
                        [-0.3147,  0.1626,  0.8728, -0.0556]])

In [4]:
t.a

tensor([[ 2.0032,  0.9398,  0.1247],
        [-0.3039,  0.9653,  0.4137]])

In [5]:
%timeit t.a

88.8 ns ± 0.919 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f51adf575e0>
├── 'a' --> tensor([[-0.3251,  0.0214, -0.2442],
│                   [-1.9965, -1.8439, -1.8390]])
└── 'x' --> <FastTreeValue 0x7f51adf57160>
    └── 'c' --> tensor([[-0.4802, -0.4363,  1.1402, -0.0180],
                        [-1.7043, -1.2722, -1.3475, -1.8720],
                        [-0.3147,  0.1626,  0.8728, -0.0556]])

In [7]:
%timeit t.a = new_value

89.9 ns ± 0.912 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 2.0032,  0.9398,  0.1247],
               [-0.3039,  0.9653,  0.4137]]),
    x: Batch(
           c: tensor([[-0.4802, -0.4363,  1.1402, -0.0180],
                      [-1.7043, -1.2722, -1.3475, -1.8720],
                      [-0.3147,  0.1626,  0.8728, -0.0556]]),
       ),
)

In [10]:
b.a

tensor([[ 2.0032,  0.9398,  0.1247],
        [-0.3039,  0.9653,  0.4137]])

In [11]:
%timeit b.a

75.7 ns ± 0.594 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.0523, -0.8453, -0.0454],
               [ 0.3319,  0.2453,  0.6680]]),
    x: Batch(
           c: tensor([[-0.4802, -0.4363,  1.1402, -0.0180],
                      [-1.7043, -1.2722, -1.3475, -1.8720],
                      [-0.3147,  0.1626,  0.8728, -0.0556]]),
       ),
)

In [13]:
%timeit b.a = new_value

636 ns ± 7.17 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

1.01 µs ± 18 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

12.2 µs ± 131 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

162 µs ± 1.34 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

165 µs ± 887 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f50e64ebc70>
├── 'a' --> tensor([[[ 2.0032,  0.9398,  0.1247],
│                    [-0.3039,  0.9653,  0.4137]],
│           
│                   [[ 2.0032,  0.9398,  0.1247],
│                    [-0.3039,  0.9653,  0.4137]],
│           
│                   [[ 2.0032,  0.9398,  0.1247],
│                    [-0.3039,  0.9653,  0.4137]],
│           
│                   [[ 2.0032,  0.9398,  0.1247],
│                    [-0.3039,  0.9653,  0.4137]],
│           
│                   [[ 2.0032,  0.9398,  0.1247],
│                    [-0.3039,  0.9653,  0.4137]],
│           
│                   [[ 2.0032,  0.9398,  0.1247],
│                    [-0.3039,  0.9653,  0.4137]],
│           
│                   [[ 2.0032,  0.9398,  0.1247],
│                    [-0.3039,  0.9653,  0.4137]],
│           
│                   [[ 2.0032,  0.9398,  0.1247],
│                    [-0.3039,  0.9653,  0.4137]]])
└── 'x' --> <FastTreeValue 0x7f50de125f40>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

37.7 µs ± 1.05 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f50de0fdbe0>
├── 'a' --> tensor([[ 2.0032,  0.9398,  0.1247],
│                   [-0.3039,  0.9653,  0.4137],
│                   [ 2.0032,  0.9398,  0.1247],
│                   [-0.3039,  0.9653,  0.4137],
│                   [ 2.0032,  0.9398,  0.1247],
│                   [-0.3039,  0.9653,  0.4137],
│                   [ 2.0032,  0.9398,  0.1247],
│                   [-0.3039,  0.9653,  0.4137],
│                   [ 2.0032,  0.9398,  0.1247],
│                   [-0.3039,  0.9653,  0.4137],
│                   [ 2.0032,  0.9398,  0.1247],
│                   [-0.3039,  0.9653,  0.4137],
│                   [ 2.0032,  0.9398,  0.1247],
│                   [-0.3039,  0.9653,  0.4137],
│                   [ 2.0032,  0.9398,  0.1247],
│                   [-0.3039,  0.9653,  0.4137]])
└── 'x' --> <FastTreeValue 0x7f50de0fd970>
    └── 'c' --> tensor([[-0.4802, -0.4363,  1.1402, -0.0180],
                        [-1.7043, -1.2722, -1.3475, -1.8720],
                 

In [23]:
%timeit t_cat(trees)

35.7 µs ± 255 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

75.9 µs ± 947 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 2.0032,  0.9398,  0.1247],
                [-0.3039,  0.9653,  0.4137]],
       
               [[ 2.0032,  0.9398,  0.1247],
                [-0.3039,  0.9653,  0.4137]],
       
               [[ 2.0032,  0.9398,  0.1247],
                [-0.3039,  0.9653,  0.4137]],
       
               [[ 2.0032,  0.9398,  0.1247],
                [-0.3039,  0.9653,  0.4137]],
       
               [[ 2.0032,  0.9398,  0.1247],
                [-0.3039,  0.9653,  0.4137]],
       
               [[ 2.0032,  0.9398,  0.1247],
                [-0.3039,  0.9653,  0.4137]],
       
               [[ 2.0032,  0.9398,  0.1247],
                [-0.3039,  0.9653,  0.4137]],
       
               [[ 2.0032,  0.9398,  0.1247],
                [-0.3039,  0.9653,  0.4137]]]),
    x: Batch(
           c: tensor([[[-0.4802, -0.4363,  1.1402, -0.0180],
                       [-1.7043, -1.2722, -1.3475, -1.8720],
                       [-0.3147,  0.1626,  0.8728, -0.0556]],
         

In [26]:
%timeit Batch.stack(batches)

96.2 µs ± 1.03 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 2.0032,  0.9398,  0.1247],
               [-0.3039,  0.9653,  0.4137],
               [ 2.0032,  0.9398,  0.1247],
               [-0.3039,  0.9653,  0.4137],
               [ 2.0032,  0.9398,  0.1247],
               [-0.3039,  0.9653,  0.4137],
               [ 2.0032,  0.9398,  0.1247],
               [-0.3039,  0.9653,  0.4137],
               [ 2.0032,  0.9398,  0.1247],
               [-0.3039,  0.9653,  0.4137],
               [ 2.0032,  0.9398,  0.1247],
               [-0.3039,  0.9653,  0.4137],
               [ 2.0032,  0.9398,  0.1247],
               [-0.3039,  0.9653,  0.4137],
               [ 2.0032,  0.9398,  0.1247],
               [-0.3039,  0.9653,  0.4137]]),
    x: Batch(
           c: tensor([[-0.4802, -0.4363,  1.1402, -0.0180],
                      [-1.7043, -1.2722, -1.3475, -1.8720],
                      [-0.3147,  0.1626,  0.8728, -0.0556],
                      [-0.4802, -0.4363,  1.1402, -0.0180],
                      [-1.7043, -

In [28]:
%timeit Batch.cat(batches)

173 µs ± 1.27 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

405 µs ± 5.78 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
